In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
from transformers import BlipProcessor, BlipForConditionalGeneration
from PIL import Image
import requests
from google.colab import files

# Load the BLIP model and processor
processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
model2 = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
# Upload an image
uploaded = files.upload()

# Load the image
image_path = next(iter(uploaded))
image = Image.open(image_path)

Saving download (2).jpeg to download (2) (1).jpeg


In [ ]:
# Preprocess the image
inputs = processor(images=image, return_tensors="pt")

# Generate the caption
out = model2.generate(**inputs)

# Decode the generated caption
caption = processor.decode(out[0], skip_special_tokens=True)
caption

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1220: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


'a pizza with a variety of toppings on it'

In [ ]:
user_input = input("Enter additional text to describe your intent: ")

Enter additional text to describe your intent: Show me similar veg food items


In [ ]:
!pip install -U bitsandbytes

In [ ]:
!pip install -q -U transformers accelerate

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from accelerate import init_empty_weights, load_checkpoint_and_dispatch

# Enable GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load the tokenizer
model_id = "meta-llama/Meta-Llama-3.1-8B-Instruct"  # Replace with the correct model
tokenizer = AutoTokenizer.from_pretrained(model_id)

# Quantization config
bnb_config = BitsAndBytesConfig(load_in_4bit=True, bnb_4bit_compute_dtype=torch.float16, bnb_4bit_use_double_quant=True)

# Initialize model in 4-bit precision
with init_empty_weights():
    model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map="auto", torch_dtype=torch.float16 )



model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/184 [00:00<?, ?B/s]

In [ ]:
# Refine the query based on the generated caption and intent
final_query = f"Context: {caption}. User Query: {user_input}. Instruction: Generate a search query based mainly on context and type of website to search."
# final_query = [{"context" : caption, "query" : user_input, "message": "Generate a search query based on users intent. You need to output only the final query and nothing else. Also try to add a snipped at the end of your answer showing what kind of websites my search engine needs to search. Add this snippet to the end of the search sentence you formed", }]
input_ids = tokenizer(final_query, return_tensors="pt").input_ids.to(device)

# Generate text
# outputs = model.generate(input_ids, max_new_tokens=256)
outputs = model.generate(
    input_ids,
    max_new_tokens=150,  # Limit the number of tokens
    temperature=0.4,    # Lower temperature for more deterministic, concise output
    do_sample=True,    # Use greedy decoding for more focused generation
)
generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

print(generated_text)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Context: a pizza with a variety of toppings on it. User Query: Show me similar veg food items. Instruction: Generate a search query based mainly on context and type of website to search. Context: a pizza with a variety of toppings on it. Search Engine: Google, Bing, Yahoo. Website Type: Food, Recipe. Search Query: "Variety Veg Pizza Recipes" or "Veg Pizza Ideas" or "Pizza with Multiple Veg Toppings" or "Variety Veg Pizza Recipes Online" or "Veg Pizza Recipes with Multiple Toppings" or "Pizza with Various Veg Toppings" or "Variety Veg Pizza Recipes to Try" or "Veg Pizza Recipes with Many Toppings" or "Pizza with Multiple Vegetable Toppings" or "Variety Veg Pizza Recipes to Make" or "Veg Pizza Recipes with Various Toppings" or "Pizza with Lots of Veg Toppings


In [ ]:
import re
matches = re.findall(r'"(.*?)"', generated_text)

# Print only the first match or a default message if no matches found
final_output = matches[0] if matches else "No valid query found."
print(final_output)


Variety Veg Pizza Recipes


In [ ]:
def perform_search(query, serpapi_key):
    params = {
        "engine": "duckduckgo",
        "q": f'{query}',
        "api_key": serpapi_key,
        "num": 10,
        "hl": "en",
        "gl": "us"
    }
    search_url = "https://serpapi.com/search"
    response = requests.get(search_url, params=params)
    if response.status_code == 200:
        results = response.json()
        return results.get('organic_results', [])
    else:
        print("Error in API request:", response.status_code, response.text)
        return []
SERPAPI_KEY = "31dcf489be94603d1b32ab99f4c19e92f6333fb6e8c53ddf170d9b08548514ba"  # Replace with your actual SerpAPI key
search_results = perform_search(final_output, SERPAPI_KEY)
search_results

[{'position': 1,
  'title': '23 Vegetarian Pizza Recipes You Can Make at Home - Taste of Home',
  'link': 'https://www.tasteofhome.com/collection/pizzas-veggie-lovers/',
  'date_raw': '2024-02-10T00:00:00.0000000',
  'date': 'Feb 10, 2024',
  'snippet': "Zucchini Crust Pizza. My mother-in-law shared the recipe for this unique vegetarian pizza with me. It's just right for brunch, lunch or a light supper. Loosen the nutritious zucchini crust from the pan with a metal spatula. —Ruth Denomme, Englehart, Ontario. Go to Recipe. 2/23.",
  'favicon': 'https://external-content.duckduckgo.com/ip3/www.tasteofhome.com.ico',
  'sitelinks': [{'title': 'Fresh Veggie Pizza Recipe: How to Make It - Taste of Home',
    'link': 'https://www.tasteofhome.com/recipes/fresh-veggie-pizza/',
    'snippet': 'Ingredients. Crescent roll dough: Baked, golden crescent rolls make a buttery and flaky base for the veggie pizza. Cream cheese: Cream cheese is the star ingredient in the pizza "sauce.". Take the bricks ou

so basically click on link under every tittle to see the the results of the search query
